##Generative AI : summarize Dialogues of DiologueSum dataset
#####Exploring different input texts and their impact on output by prompt engineering, zero shot, one shot & few shot inferences.
#####At then end, exploration of different configuration parameters and their impact on outputs

###1. Installing Python `libraries`

In [0]:
!pip install --disable-pip-version-check torch

Collecting torch
Successfully installed fsspec-2024.6.1 mpmath-1.3.0 networkx-3.1 nvidia-cublas-cu12-12.1.3.1 nvidia-cuda-cupti-cu12-12.1.105 nvidia-cuda-nvrtc-cu12-12.1.105 nvidia-cuda-runtime-cu12-12.1.105 nvidia-cudnn-cu12-8.9.2.26 nvidia-cufft-cu12-11.0.2.54 nvidia-curand-cu12-10.3.2.106 nvidia-cusolver-cu12-11.4.5.107 nvidia-cusparse-cu12-12.1.0.106 nvidia-nccl-cu12-2.20.5 nvidia-nvjitlink-cu12-12.5.82 nvidia-nvtx-cu12-12.1.105 sympy-1.13.0 torch-2.3.1 triton-2.3.1 typing-extensions-4.12.2

In [0]:
!pip install --disable-pip-version-check torchdata

Collecting torchdata
Successfully installed torchdata-0.7.1

In [0]:
!pip install transformers

Collecting transformers
 Downloading transformers-4.42.4-py3-none-any.whl (9.3 MB)
Successfully installed huggingface-hub-0.23.4 pyyaml-6.0.1 regex-2024.5.15 safetensors-0.4.3 tokenizers-0.19.1 tqdm-4.66.4 transformers-4.42.4

In [0]:
!pip install datasets==2.18.0 --quiet

WARNING: You are using pip version 21.0.1; however, version 24.1.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import datasets
print(datasets.__version__)

2.18.0

In [0]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

###2. Summarize Dialogue without Prompt engineering

In [0]:
hugging_dataset = 'knkarthick/dialogsum'
dataset = load_dataset(hugging_dataset)

Downloading readme: 0%| | 0.00/4.65k [00:00<?, ?B/s]
Downloading data: 0%| | 0.00/11.3M [00:00<?, ?B/s]Downloading data: 93%|█████████▎| 10.5M/11.3M [00:00<00:00, 46.0MB/s]Downloading data: 100%|██████████| 11.3M/11.3M [00:00<00:00, 46.8MB/s]
Downloading data: 0%| | 0.00/442k [00:00<?, ?B/s]Downloading data: 100%|██████████| 442k/442k [00:00<00:00, 2.76MB/s]Downloading data: 100%|██████████| 442k/442k [00:00<00:00, 2.75MB/s]
Downloading data: 0%| | 0.00/1.35M [00:00<?, ?B/s]Downloading data: 100%|██████████| 1.35M/1.35M [00:00<00:00, 9.90MB/s]Downloading data: 100%|██████████| 1.35M/1.35M [00:00<00:00, 9.78MB/s]
Generating train split: 0 examples [00:00, ? examples/s]
Generating validation split: 0 examples [00:00, ? examples/s]
Generating test split: 0 examples [00:00, ? examples/s]

In [0]:
### Reading sample of dialogues
sample_indices = [40, 200]
dash_line = '_'.join('' for x in range(100))
for i, index in enumerate(sample_indices):
  print(dash_line)
  print('sample ', i+1)
  print(dash_line)
  print('Input dialogue:')
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print('human summary:')
  print(dataset['test'][index]['summary'])
  print(dash_line)
  print()

___________________________________________________________________________________________________
sample 1
___________________________________________________________________________________________________
Input dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________

___________________________________________________________________________________________________
sample 2
___________________________________________________________________________________________________
Input dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
___________________________________________________________________________________________________
human summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________

In [0]:
## Reading Flan T5 model
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [0]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

tokenizer_config.json: 0%| | 0.00/2.54k [00:00<?, ?B/s]
spiece.model: 0%| | 0.00/792k [00:00<?, ?B/s]
tokenizer.json: 0%| | 0.00/2.42M [00:00<?, ?B/s]
special_tokens_map.json: 0%| | 0.00/2.20k [00:00<?, ?B/s]

In [0]:
## sample of encoded sentence and decoded vector of the sentence
sentence = "What time is it, Matt?"
sentence_encoded = tokenizer(sentence, return_tensors='pt')
sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0], skip_special_tokens=True)
print('Encoded sentence : ')
print(sentence_encoded['input_ids'][0])
print('\n decoded sentence: ')
print(sentence_decoded)

Encoded sentence : 
tensor([ 363, 97, 19, 34, 6, 5199, 58, 1])

 decoded sentence: 
What time is it, Matt?

In [0]:
for i, index in enumerate(sample_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  inputs = tokenizer(dialogue, return_tensors='pt')
  output = tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens=50,)[0], skip_special_tokens=True)

  print(dash_line)
  print('Sample ', i+1)
  print(dash_line)
  print('Input prompt: \n', dialogue)
  print(dash_line)
  print('Baseline human summary: \n', summary)
  print(dash_line)
  print('Model generation - without prompt engineering: \n', output)

___________________________________________________________________________________________________
Sample 1
___________________________________________________________________________________________________
Input prompt: 
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
Baseline human summary: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
Model generation - without prompt engineering: 
 Person1: It's ten to nine.
___________________________________________________________________________________________________
Sample 2
___________________________________________________________________________________________________
Input prompt: 
 #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
___________________________________________________________________________________________________
Baseline human summary: 
 #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
Model generation - without prompt engineering: 
 #Person1#: I'm thinking of upgrading my computer.

##### Observation : Not a good quality of text summarization

###3. Zero shot inference with an instruction prompt

In [0]:
for i, index in enumerate(sample_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  Summarize the following conversation.
  {dialogue}
  Summary :
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens=50,)[0], skip_special_tokens=True)

  print(dash_line)
  print('Sample ', i+1)
  print(dash_line)
  print('Input prompt: \n', prompt)
  print(dash_line)
  print('Baseline human summary: \n', summary)
  print(dash_line)
  print('Model generation - without prompt engineering: \n', output)

___________________________________________________________________________________________________
Sample 1
___________________________________________________________________________________________________
Input prompt: 
 
 Summarize the following conversation.
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
 Summary :
 
___________________________________________________________________________________________________
Baseline human summary: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
Model generation - without prompt engineering: 
 The train is about to leave.
___________________________________________________________________________________________________
Sample 2
___________________________________________________________________________________________________
Input prompt: 
 
 Summarize the following conversation.
 #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
 Summary :
 
___________________________________________________________________________________________________
Baseline human summary: 
 #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
Model generation - without prompt engineering: 
 #Person1#: I'm thinking of upgrading my computer.

In [0]:
for i, index in enumerate(sample_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
 Dialogue:
  {dialogue}
  What was going on ?
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens=50,)[0], skip_special_tokens=True)

  print(dash_line)
  print('Sample ', i+1)
  print(dash_line)
  print('Input prompt: \n', prompt)
  print(dash_line)
  print('Baseline human summary: \n', summary)
  print(dash_line)
  print('Model generation - without prompt engineering: \n', output)

___________________________________________________________________________________________________
Sample 1
___________________________________________________________________________________________________
Input prompt: 
 
 Dialogue:
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
 What was going on ?
 
___________________________________________________________________________________________________
Baseline human summary: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
Model generation - without prompt engineering: 
 Tom is late for the train.
___________________________________________________________________________________________________
Sample 2
___________________________________________________________________________________________________
Input prompt: 
 
 Dialogue:
 #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
 What was going on ?
 
___________________________________________________________________________________________________
Baseline human summary: 
 #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
Model generation - without prompt engineering: 
 #Person1#: You might consider upgrading your system. #Person2#: You might also consider adding a painting program to your software. #Person1#: You might also consider adding a faster processor, more memory

##### Observation : Not a good quality of text summarization

###4. One shot inference

In [0]:
def make_promt(sample_indices, sample_index_to_summarize):
  prompt = ''
  for index in sample_indices:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    prompt += f"""
    Dialogue:
    {dialogue}
    What was going on?
    {summary}
    """
  dialogue = dataset['test'][sample_index_to_summarize]['dialogue']
  prompt += f"""
  Dialogue:
  {dialogue}
  What was going on?
  """
  return prompt

In [0]:
sample_indices = [40]
sample_index_to_summarize = 200

one_shot_prompt = make_promt(sample_indices, sample_index_to_summarize)
print(one_shot_prompt)

Dialogue:
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
 What was going on?
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
 
 Dialogue:
 #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
 What was going on?

In [0]:
summary = dataset['test'][sample_index_to_summarize]['summary']
inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens = 50)[0], skip_special_tokens = True)
print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - one shot : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - one shot : {'#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.'}

##### Observation : Better quality of text summarization and more specific in comparsion to zero shot

###5. Few shot inference

In [0]:
### Adding three shots
sample_indices = [40, 80, 120]
sample_index_to_summarize = 200
few_shot_prompt = make_promt(sample_indices, sample_index_to_summarize)
print(few_shot_prompt)

Dialogue:
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
 What was going on?
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
 
 Dialogue:
 #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. May, can you help me take all these things to the living room?
#Person2#: Yes, madam.
#Person1#: Ask Daniel to give you a hand?
#Person2#: No, mom, I can manage it by myself. His help just causes more trouble.
 What was going on?
 Mom asks May to help to prepare for the picnic and May agrees.
 
 Dialogue:
 #Person1#: Hello, I bought the pendant in your shop, just before. 
#Person2#: Yes. Thank you very much. 
#Person1#: Now I come back to the hotel and try to show it to my friend, the pendant is broken, I'm afraid. 
#Person2#: Oh, is it? 
#Person1#: Would you change it to a new one? 
#Person2#: Yes, certainly. You have the receipt? 
#Person1#: Yes, I do. 
#Person2#: Then would you kindly come to our shop with the receipt by 10 o'clock? We will replace it. 
#Person1#: Thank you so much. 
 What was going on?
 #Person1# wants to change the broken pendant in #Person2#'s shop.
 
 Dialogue:
 #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
 What was going on?

In [0]:
summary = dataset['test'][sample_index_to_summarize]['summary']
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], max_new_tokens = 50)[0], skip_special_tokens = True)
print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.'}

##### Observation : Indeed, btter quality of text summarization in comparison to zero shot, but not that difference in comparsino to one shot

#### Conclusion on feeding shots : adding shots to the model could make the summary better, but still not as expected

###6. Generative configuration Parameters for Inference

In [0]:
generation_config = GenerationConfig(max_new_tokens = 50)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.'}

In [0]:
### decreasing the max of output text by changing the nax_new_tokens value
generation_config = GenerationConfig(max_new_tokens = 30)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants'}

In [0]:
### decreasing the max of output text by changing the nax_new_tokens value
generation_config = GenerationConfig(max_new_tokens = 10)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 wants to upgrade his system.'}

#### Conclusion of changing the max token: 
#####This is used for max size of output sequence, not including the tokens in the prompt. Btter to tune which length of output text is desirable by end users without limiting valuable data.

In [0]:
generation_config = GenerationConfig(max_new_tokens = 50, do_sample= True, temperature = 0.1)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.'}

In [0]:
### Chaning the temperature parameter to get away from default Gready search approach
generation_config = GenerationConfig(max_new_tokens = 50, do_sample= True, temperature = 0.5)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 offers to upgrade his system to a painting program.'}

In [0]:
### Chaning the temperature parameter to get away from default Gready search approach
generation_config = GenerationConfig(max_new_tokens = 50, do_sample= True, temperature = 1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'"I\'m quite sure I would require your services" with regard to software upgrades.'}

#### Conclusion of changing the temperature parameter: 
#####increasing the temperature value triggers creative responses. But decreasing the temperature triggers conservative reponses which means by runing the code over and over again, the same response will be made. This happens because of default greedy search approach in text generation which always choose the text with highest probability. This parameter should not get too high, otherwise it makes semantic inconsistency

In [0]:
## Greedy search as default search
generation_config = GenerationConfig(max_new_tokens = 50)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.'}

In [0]:
## multinomial sampling
generation_config = GenerationConfig(max_new_tokens = 50, do_sample=True, num_beams=1)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'People are discussing about upgrading a computer. They want to make money from the upgrade; they want a computer that will be more user-friendly and faster. They want to have a CD-ROM drive and better hardware.'}

In [0]:
## Beam search
generation_config = GenerationConfig(max_new_tokens = 50, num_beams=5)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 recommends adding a painting program to the software. #Person2 recommends adding a CD-ROM drive.'}

In [0]:
## Beam search multinomial sampling
generation_config = GenerationConfig(max_new_tokens = 50, num_beams=5, do_sample=True)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware. #Person2 wants to add a CD-ROM drive.'}

In [0]:
## Contrastive search
generation_config = GenerationConfig(max_new_tokens = 50, penalty_alpha=0.6, top_k=4)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)

print(dash_line)
print('baseline human summary', {summary})
print(dash_line)
print('model generation - few shots : ', {output})

___________________________________________________________________________________________________
baseline human summary {"#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."}
___________________________________________________________________________________________________
model generation - few shots : {'#Person1 suggests adding a painting program to the software. #Person2 suggests adding a computer graphics program.'}

#### Conclusion of different text generation methods: 
#####Deterministic approaches such as greedy search & beam search result in text degenration & undesirebale repition. Though, the stochastic approaches are using randomness during text generation process which end with non-repetitive yet coherent long outputs
more info about text generation methods in below link : https://huggingface.co/docs/transformers/generation_strategies